In [3]:
import pandas as pd
from unidecode import unidecode
import numpy as np
import scipy.io
from scipy.optimize import minimize
from scipy.stats import norm
import matplotlib.pyplot as plt
import os

PATH_UPV = "./DatosExperimento1/UPV"

# Leer el archivo Excel en un DataFrame de pandas
df = pd.read_excel(PATH_UPV+'/Exp1_Observadores_UPV.xlsx')
    
if not os.path.exists('Resultados_UPV'):
    os.makedirs('Resultados_UPV')

# Crear un diccionario vacío para almacenar los resultados
voluntarios_upv = {}

# Iterar sobre cada fila del DataFrame
for idx, row in df.iterrows():
    # Tratamiento de nombre y apellidos para evitar que una misma persona se trate como diferente:
    # Quito espacios, mayusculas, minuscula s, etc
    nombre = unidecode(row['Nombre'].lower())
    apellidos = unidecode(row['Apellidos'].lower())
    fichero = row['Fichero']
    # Crear la clave del diccionario usando el nombre y apellidos
    clave = f"{nombre}-{apellidos}".replace(" ", "")
    
    # Agregar el valor de la columna "Fichero" a la lista correspondiente en el diccionario
    if clave in voluntarios_upv:
        voluntarios_upv[clave].append(fichero)
    else:
        voluntarios_upv[clave] = [fichero]
voluntarios_upv

{'khleef-almutairi': ['ResultsJNDsNoiseFlatBG20211129T112353.mat',
  'ResultsJNDsNoiseFlatBG20211129T120005.mat'],
 'juan-giner': ['ResultsJNDsNoiseFlatBG20211129T133717.mat',
  'ResultsJNDsNoiseFlatBG20211129T140322.mat'],
 'sergio-camp': ['ResultsJNDsNoiseFlatBG20211129T163621.mat',
  'ResultsJNDsNoiseFlatBG20211129T171934.mat'],
 'antonio-hervasjorge': ['ResultsJNDsNoiseFlatBG20211130T105700.mat'],
 'jessica-corderoruiz': ['ResultsJNDsNoiseFlatBG20211201T114113.mat'],
 'josealberto-conejero': ['ResultsJNDsNoiseFlatBG20211202T140213.mat'],
 'josealberto-conejerocasares': ['ResultsJNDsNoiseFlatBG20211202T142641.mat'],
 'mariajose-rodriguezalvarez': ['ResultsJNDsNoiseFlatBG20211203T102719.mat',
  'ResultsJNDsNoiseFlatBG20220124T090423.mat'],
 'cristina-martinezvallier': ['ResultsJNDsNoiseFlatBG20211210T101730.mat',
  'ResultsJNDsNoiseFlatBG20211210T112932.mat'],
 'laura-mislatalahiguera': ['ResultsJNDsNoiseFlatBG20211210T105955.mat'],
 'jessica-cordero': ['ResultsJNDsNoiseFlatBG2021121

In [4]:
def normcdf2(x, mu, sigma):
    '''
    Utilizo la función norm.cdf de la librería scipy.stats para calcular la función de distribución acumulada (CDF) 
    de la distribución normal, y luego dividimos el resultado por dos y sumamos 0.5 para ajustar el rango a [0,1].
    '''
    y = (norm.cdf(x, mu, sigma) + 1) / 2
    return y

In [5]:
def normcdf2_minimizer(X, params):
    # X incluye mu y sigma
    mu = X[0]
    sigma = X[1]

    # x/y valores para ajustar
    x = params['x']
    y = params['y']

    # los valores ajustados con mu y sigma dados
    yfit = normcdf2(x, mu, sigma)

    # cálculo del error
    err = np.sum((yfit - y)**2)

    return err

In [39]:
# Variables generales
ruidos = scipy.io.loadmat('ruidosExp1.mat')
ruidos = np.array(ruidos['ruidos'])
minX = np.min(ruidos[:, 2])
maxX = np.max(ruidos[:, 2])
LoopNumber = 16
numberObservations = 10
    
for voluntario, ficheros in voluntarios_upv.items():    
    mean_general_sin_voluntario = None
    num_ficheros = 0
    
    # Calculo de la media general sin el voluntario actual
    for vol, fichs in voluntarios_upv.items():
        if vol!=voluntario:
            for fic in fichs:
                data_file = PATH_UPV+'/'+fic
                result = scipy.io.loadmat(data_file)
                result = np.array(result['Results'])
                if mean_general_sin_voluntario is None:
                    mean_general_sin_voluntario = result
                else:
                    mean_general_sin_voluntario = mean_general_sin_voluntario + result 
                    # Suma de los resultados
                    # Promediar la probabilidad y Promediar el tiempo
                num_ficheros+=len(fichs)
    mean_general_sin_voluntario = mean_general_sin_voluntario/num_ficheros
    
    
    # Calculo de la media del voluntario
    mean_voluntario = None
    for fichero in ficheros:
        data_file = PATH_UPV+'/'+fichero
        result = scipy.io.loadmat(data_file)
        result = np.array(result['Results'])
        if mean_voluntario is None:
            mean_voluntario = result
        else:
            mean_voluntario = mean_voluntario + result 
            # Suma de los resultados
            # Promediar la probabilidad y Promediar el tiempo
    mean_voluntario = mean_voluntario/len(ficheros) # Media de los resultados para un mismo voluntario
    
    # Calculo el rango esperado
    std_general = [np.std(mean_general_sin_voluntario[:,0]), np.std(mean_general_sin_voluntario[:,1])]
    lower_bound = mean_general_sin_voluntario - 2 * np.array(std_general).reshape((1, 2))
    upper_bound = mean_general_sin_voluntario + 2 * np.array(std_general).reshape((1, 2))
    
    # Comparar la media del voluntario actual con la media general y detectar outliers
    # Realiza los gráficos si es outlier
    experimento_outlier = (mean_voluntario < lower_bound) | (mean_voluntario > upper_bound)
    # print("Ambas columnas son outliers\n")
    ambas = np.where(experimento_outlier.all(axis=1))[0]
    # print(ambas) # Ambos son Outlier
    # print("Al menos una son outliers\n")
    al_menos_una = np.where(experimento_outlier)[0]
    # print(al_menos_una) # Al menos un es Outlier
    
    for i in range(160):
        print(str(mean_voluntario[i])+'\n'+str(mean_general_sin_voluntario[i])+"\n----------------------------\n")
    
    if (len(ambas)>3):
        '''
        print(f"El voluntario {voluntario} es un outlier.")
        print(f"Sus experimentos outliers son: {ambas}.")
        print("Valores en esos experimentos:")
        print([mean_voluntario[i-1] for i in ambas])
        print("Y la del resto de voluntarios_upv:")
        print([mean_general_sin_voluntario[i-1] for i in ambas])
            
        if not os.path.exists('./Resultados_UPV/0-OUTLIERS'):
            os.makedirs('./Resultados_UPV/0-OUTLIERS')
            
        if not os.path.exists('./Resultados_UPV/0-OUTLIERS/'+voluntario):
            os.makedirs('./Resultados_UPV/0-OUTLIERS/'+voluntario)

        results_path = './Resultados_UPV/0-OUTLIERS/'+voluntario+'/'
        # Individual plot
        for i in range(LoopNumber):
            x = ruidos[i*numberObservations:(i+1)*numberObservations, 2]
            y = mean_voluntario[i*numberObservations:(i+1)*numberObservations, 0]

            # Valores iniciales de la minimización
            X0 = [np.mean(x), 0.1*np.mean(x)]

            # parámetros para el ajuste
            params = {'x': x, 'y': y}

            # minimización
            ParamsOpt = minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x

            # media y la desviación estándar de la distribución ajustada
            meanAndStdDistrib = [ParamsOpt[0], ParamsOpt[1]]

            # resultados en gráficos independientes
            plt.figure(i)
            plt.plot(x, y, 'o--b')
            plt.axis([min(x), max(x), 0, 1])
            x_curve = np.linspace(minX, maxX, num=1000)
            y_curve = normcdf2(x_curve, ParamsOpt[0], ParamsOpt[1])
            plt.plot(x_curve, y_curve, '-r', linewidth=2)
            plt.title('Psychometric Curve for '+voluntario+ ' ('+ str(i+1)+")")
            plt.xlabel('Noise value')
            plt.ylabel('Detection probability')
            plt.legend(['Experimental values', 'Psychometrically fitted curve'], loc='lower right')
            plt.grid(True)
            plt.xlim([minX, maxX])
            plt.ylim([0, 1])

            # Guardar el gráfico y los datos en .npy
            plt.savefig(results_path+voluntario+'-plot-' + str(i+1) + '.png')
            np.save(results_path+voluntario+'-MeanStdDistribution-' + str(i+1) + '.npy', meanAndStdDistrib)

            plt.show()


        # 4 times 4 psychophysics plot

        plt.figure() 
        figure = plt.figure(figsize=(numberObservations, numberObservations))
        figure.subplots_adjust(hspace=0.5, wspace=0.5)

        for i in range(LoopNumber):
            x = ruidos[i*numberObservations:(i+1)*numberObservations, 2]
            y = mean_voluntario[i*numberObservations:(i+1)*numberObservations, 0]

            # Valores iniciales de la minimización
            X0 = [np.mean(x), 0.1*np.mean(x)]

            # Definir los parámetros para el ajuste
            params = {'x': x, 'y': y}

            # Realizar la minimización
            ParamsOpt = minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x

            # Obtener la media y la desviación estándar de la distribución ajustada
            meanAndStdDistrib = [ParamsOpt[0], ParamsOpt[1]]

            ax = figure.add_subplot(4, 4, i+1)
            ax.plot(x, y, 'o--b')
            x_curve = np.linspace(minX, maxX, num=1000)
            y_curve = normcdf2(x_curve, ParamsOpt[0], ParamsOpt[1])
            ax.plot(x_curve, y_curve, '-r', linewidth=2)
            ax.grid(True)
            ax.set_xlim([minX, maxX])
            ax.set_ylim([0, 1])
            ax.set_title(str(i+1))
            ax.set_xlabel('Noise value')
            ax.set_ylabel('Detection probability')

        figure.suptitle('4\\times4 psychophysics plot for '+voluntario)
        plt.savefig(results_path+voluntario+'-MosaicWithEqualXAxes-' + str(i+1) + '.png')


        # All 16 psychophysical curves

        plt.figure() 
        cc = plt.get_cmap('jet')(np.linspace(0, 1, 16))

        fig, ax = plt.subplots()
        for i in range(LoopNumber):
            x = ruidos[i*numberObservations:(i+1)*numberObservations, 2]
            y = mean_voluntario[i*numberObservations:(i+1)*numberObservations, 0]

            # Valores iniciales de la minimización
            X0 = [np.mean(x), 0.1*np.mean(x)]

            # Definir los parámetros para el ajuste
            params = {'x': x, 'y': y}

            # Realizar la minimización
            ParamsOpt = minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x

            # Obtener la media y la desviación estándar de la distribución ajustada
            meanAndStdDistrib = [ParamsOpt[0], ParamsOpt[1]]

            x_curve = np.linspace(minX, maxX, num=1000)
            y_curve = normcdf2(x_curve, ParamsOpt[0], ParamsOpt[1])
            ax.plot(x_curve, y_curve, linewidth=2, color=cc[i])
            ax.grid(True)
            ax.set_xlim([minX, maxX])
            ax.set_ylim([0, 1])
            ax.set_xlabel('Noise value')
            ax.set_ylabel('Detection probability')

        ax.set_title('All 16 psychophysical curves for '+voluntario)

        Legend = []
        for i in range(16):
            Legend.append('4x4:{}'.format(i+1))

        ax.legend(Legend)

        plt.savefig(results_path+voluntario+'-AllPsychomPlots-' + str(i+1) + '.png')
        
        '''

IndentationError: expected an indented block after 'for' statement on line 59 (1984512199.py, line 62)

In [40]:
PATH_UGR = "./DatosExperimento1/UGR"

# Leer el archivo Excel en un DataFrame de pandas
df = pd.read_excel(PATH_UGR+'/Exp1_Observadores_UGR.xlsx')
    
if not os.path.exists('Resultados_UGR'):
    os.makedirs('Resultados_UGR')

# Crear un diccionario vacío para almacenar los resultados
voluntarios_ugr = {}

# Iterar sobre cada fila del DataFrame
for idx, row in df.iterrows():
    # Tratamiento de nombre y apellidos para evitar que una misma persona se trate como diferente:
    # Quito espacios, mayusculas, minuscula s, etc
    nombre = unidecode(row['Nombre'].lower())
    apellidos = unidecode(row['Apellidos'].lower())
    fichero = row['Fichero']
    # Crear la clave del diccionario usando el nombre y apellidos
    clave = f"{nombre}-{apellidos}".replace(" ", "")
    
    # Agregar el valor de la columna "Fichero" a la lista correspondiente en el diccionario
    if clave in voluntarios_ugr:
        voluntarios_ugr[clave].append(fichero)
    else:
        voluntarios_ugr[clave] = [fichero]
voluntarios_ugr

{'marcos-algabamesa': ['ResultsJNDsNoiseFlatBG20220401T131441.mat'],
 'victor-callejarodriguez': ['ResultsJNDsNoiseFlatBG20220317T113653.mat',
  'ResultsJNDsNoiseFlatBG20220407T113228.mat'],
 'miriam-casareslopez': ['ResultsJNDsNoiseFlatBG20220201T134054.mat',
  'ResultsJNDsNoiseFlatBG20220202T124958.mat'],
 'josejuan-castrotorres': ['ResultsJNDsNoiseFlatBG20220427T114635.mat',
  'ResultsJNDsNoiseFlatBG20220408T103353.mat'],
 'adela-colladorodriguez': ['ResultsJNDsNoiseFlatBG20220401T121203.mat',
  'ResultsJNDsNoiseFlatBG20220323T132705.mat'],
 'maximilian-czech': ['ResultsJNDsNoiseFlatBG20220303T125835.mat',
  'ResultsJNDsNoiseFlatBG20220314T113216.mat'],
 'austin-english': ['ResultsJNDsNoiseFlatBG20220318T133520.mat'],
 'miguel-escanohuertas': ['ResultsJNDsNoiseFlatBG20220411T193944.mat',
  'ResultsJNDsNoiseFlatBG20220420T172846.mat'],
 'cristina-espinarpulgar': ['ResultsJNDsNoiseFlatBG20220223T112641.mat',
  'ResultsJNDsNoiseFlatBG20220223T161544.mat'],
 'beatriz-estebanmolina': ['R

In [41]:
# Variables generales
ruidos = scipy.io.loadmat('ruidosExp1.mat')
ruidos = np.array(ruidos['ruidos'])
minX = np.min(ruidos[:, 2])
maxX = np.max(ruidos[:, 2])
LoopNumber = 16
numberObservations = 10
    
for voluntario, ficheros in voluntarios_ugr.items():    
    mean_general_sin_voluntario = None
    num_ficheros = 0
    
    # Calculo de la media general sin el voluntario actual
    for vol, fichs in voluntarios_ugr.items():
        if vol!=voluntario:
            for fic in fichs:
                data_file = PATH_UGR+'/'+fic
                result = scipy.io.loadmat(data_file)
                result = np.array(result['Results'])
                if mean_general_sin_voluntario is None:
                    mean_general_sin_voluntario = result
                else:
                    mean_general_sin_voluntario = mean_general_sin_voluntario + result 
                    # Suma de los resultados
                    # Promediar la probabilidad y Promediar el tiempo
                num_ficheros+=len(fichs)
    mean_general_sin_voluntario = mean_general_sin_voluntario/num_ficheros
    
    
    # Calculo de la media del voluntario
    mean_voluntario = None
    for fichero in ficheros:
        data_file = PATH_UGR+'/'+fichero
        result = scipy.io.loadmat(data_file)
        result = np.array(result['Results'])
        if mean_voluntario is None:
            mean_voluntario = result
        else:
            mean_voluntario = mean_voluntario + result 
            # Suma de los resultados
            # Promediar la probabilidad y Promediar el tiempo
    mean_voluntario = mean_voluntario/len(ficheros) # Media de los resultados para un mismo voluntario
    
    # Calculo el rango esperado
    std_general = [np.std(mean_general_sin_voluntario[:,0]), np.std(mean_general_sin_voluntario[:,1])]
    lower_bound = mean_general_sin_voluntario - 2 * np.array(std_general).reshape((1, 2))
    upper_bound = mean_general_sin_voluntario + 2 * np.array(std_general).reshape((1, 2))
    
    # Comparar la media del voluntario actual con la media general y detectar outliers
    # Realiza los gráficos si es outlier
    experimento_outlier = (mean_voluntario < lower_bound) | (mean_voluntario > upper_bound)
    # print("Ambas columnas son outliers\n")
    ambas = np.where(experimento_outlier.all(axis=1))[0]
    # print(ambas) # Ambos son Outlier
    # print("Al menos una son outliers\n")
    al_menos_una = np.where(experimento_outlier)[0]
    # print(al_menos_una) # Al menos un es Outlier
    
    for i in range(160):
        print(str(mean_voluntario[i])+'\n'+str(mean_general_sin_voluntario[i])+"\n----------------------------\n")
    
    if (len(ambas)>3):
        '''
        print(f"El voluntario {voluntario} es un outlier.")
        print(f"Sus experimentos outliers son: {ambas}.")
        print("Valores en esos experimentos:")
        print([mean_voluntario[i-1] for i in ambas])
        print("Y la del resto de voluntarios_ugr:")
        print([mean_general_sin_voluntario[i-1] for i in ambas])
            
        if not os.path.exists('./Resultados_UGR/0-OUTLIERS'):
            os.makedirs('./Resultados_UGR/0-OUTLIERS')
            
        if not os.path.exists('./Resultados_UGR/0-OUTLIERS/'+voluntario):
            os.makedirs('./Resultados_UGR/0-OUTLIERS/'+voluntario)

        results_path = './Resultados_UGR/0-OUTLIERS/'+voluntario+'/'
        # Individual plot
        for i in range(LoopNumber):
            x = ruidos[i*numberObservations:(i+1)*numberObservations, 2]
            y = mean_voluntario[i*numberObservations:(i+1)*numberObservations, 0]

            # Valores iniciales de la minimización
            X0 = [np.mean(x), 0.1*np.mean(x)]

            # parámetros para el ajuste
            params = {'x': x, 'y': y}

            # minimización
            ParamsOpt = minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x

            # media y la desviación estándar de la distribución ajustada
            meanAndStdDistrib = [ParamsOpt[0], ParamsOpt[1]]

            # resultados en gráficos independientes
            plt.figure(i)
            plt.plot(x, y, 'o--b')
            plt.axis([min(x), max(x), 0, 1])
            x_curve = np.linspace(minX, maxX, num=1000)
            y_curve = normcdf2(x_curve, ParamsOpt[0], ParamsOpt[1])
            plt.plot(x_curve, y_curve, '-r', linewidth=2)
            plt.title('Psychometric Curve for '+voluntario+ ' ('+ str(i+1)+")")
            plt.xlabel('Noise value')
            plt.ylabel('Detection probability')
            plt.legend(['Experimental values', 'Psychometrically fitted curve'], loc='lower right')
            plt.grid(True)
            plt.xlim([minX, maxX])
            plt.ylim([0, 1])

            # Guardar el gráfico y los datos en .npy
            plt.savefig(results_path+voluntario+'-plot-' + str(i+1) + '.png')
            np.save(results_path+voluntario+'-MeanStdDistribution-' + str(i+1) + '.npy', meanAndStdDistrib)

            plt.show()


        # 4 times 4 psychophysics plot

        plt.figure() 
        figure = plt.figure(figsize=(numberObservations, numberObservations))
        figure.subplots_adjust(hspace=0.5, wspace=0.5)

        for i in range(LoopNumber):
            x = ruidos[i*numberObservations:(i+1)*numberObservations, 2]
            y = mean_voluntario[i*numberObservations:(i+1)*numberObservations, 0]

            # Valores iniciales de la minimización
            X0 = [np.mean(x), 0.1*np.mean(x)]

            # Definir los parámetros para el ajuste
            params = {'x': x, 'y': y}

            # Realizar la minimización
            ParamsOpt = minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x

            # Obtener la media y la desviación estándar de la distribución ajustada
            meanAndStdDistrib = [ParamsOpt[0], ParamsOpt[1]]

            ax = figure.add_subplot(4, 4, i+1)
            ax.plot(x, y, 'o--b')
            x_curve = np.linspace(minX, maxX, num=1000)
            y_curve = normcdf2(x_curve, ParamsOpt[0], ParamsOpt[1])
            ax.plot(x_curve, y_curve, '-r', linewidth=2)
            ax.grid(True)
            ax.set_xlim([minX, maxX])
            ax.set_ylim([0, 1])
            ax.set_title(str(i+1))
            ax.set_xlabel('Noise value')
            ax.set_ylabel('Detection probability')

        figure.suptitle('4\\times4 psychophysics plot for '+voluntario)
        plt.savefig(results_path+voluntario+'-MosaicWithEqualXAxes-' + str(i+1) + '.png')


        # All 16 psychophysical curves

        plt.figure() 
        cc = plt.get_cmap('jet')(np.linspace(0, 1, 16))

        fig, ax = plt.subplots()
        for i in range(LoopNumber):
            x = ruidos[i*numberObservations:(i+1)*numberObservations, 2]
            y = mean_voluntario[i*numberObservations:(i+1)*numberObservations, 0]

            # Valores iniciales de la minimización
            X0 = [np.mean(x), 0.1*np.mean(x)]

            # Definir los parámetros para el ajuste
            params = {'x': x, 'y': y}

            # Realizar la minimización
            ParamsOpt = minimize(normcdf2_minimizer, X0, args=params, method='Nelder-Mead').x

            # Obtener la media y la desviación estándar de la distribución ajustada
            meanAndStdDistrib = [ParamsOpt[0], ParamsOpt[1]]

            x_curve = np.linspace(minX, maxX, num=1000)
            y_curve = normcdf2(x_curve, ParamsOpt[0], ParamsOpt[1])
            ax.plot(x_curve, y_curve, linewidth=2, color=cc[i])
            ax.grid(True)
            ax.set_xlim([minX, maxX])
            ax.set_ylim([0, 1])
            ax.set_xlabel('Noise value')
            ax.set_ylabel('Detection probability')

        ax.set_title('All 16 psychophysical curves for '+voluntario)

        Legend = []
        for i in range(16):
            Legend.append('4x4:{}'.format(i+1))

        ax.legend(Legend)

        plt.savefig(results_path+voluntario+'-AllPsychomPlots-' + str(i+1) + '.png')
        
        '''

[0.33333333 0.97735709]
[0.28265766 1.11406175]
----------------------------

[0.66666667 0.6372431 ]
[0.32882883 1.10010846]
----------------------------

[0.66666667 2.86318479]
[0.37387387 1.11554052]
----------------------------

[0.66666667 1.48258491]
[0.40540541 1.12201818]
----------------------------

[1.         1.17687008]
[0.42117117 0.98099609]
----------------------------

[0.66666667 0.41842775]
[0.40765766 0.92432071]
----------------------------

[0.66666667 0.54437207]
[0.43468468 0.89151228]
----------------------------

[1.         0.67820382]
[0.44594595 0.88846959]
----------------------------

[0.66666667 1.23282268]
[0.45495495 0.75156094]
----------------------------

[0.66666667 0.57173355]
[0.45495495 0.69375115]
----------------------------

[0.33333333 1.1820239 ]
[0.30518018 1.1538345 ]
----------------------------

[0.66666667 0.49355061]
[0.33220721 1.08409709]
----------------------------

[0.33333333 1.57895011]
[0.36036036 1.02980231]
----------------

[0.5        1.59363204]
[0.28390805 1.12187056]
----------------------------

[0.66666667 1.49853582]
[0.33103448 1.10659464]
----------------------------

[0.66666667 1.5978543 ]
[0.37701149 1.13632741]
----------------------------

[0.66666667 2.95591442]
[0.4091954  1.11468584]
----------------------------

[0.83333333 0.93824928]
[0.42528736 0.99646753]
----------------------------

[1.         1.04200824]
[0.40689655 0.93195777]
----------------------------

[1.         1.18717624]
[0.43448276 0.89733681]
----------------------------

[1.         0.89941625]
[0.44827586 0.89912325]
----------------------------

[1.        1.0818351]
[0.45517241 0.76069084]
----------------------------

[0.66666667 0.86126498]
[0.45977011 0.70016809]
----------------------------

[1.         0.76865265]
[0.3        1.17525672]
----------------------------

[1.         1.10641691]
[0.32988506 1.09466956]
----------------------------

[0.83333333 1.02261414]
[0.35862069 1.04789285]
------------------

[1.         2.89446615]
[0.46206897 0.67092692]
----------------------------

[1.         2.87533965]
[0.46896552 0.64461322]
----------------------------

[0.83333333 1.66640157]
[0.27931034 1.12086685]
----------------------------

[0.66666667 1.72308871]
[0.33103448 1.10349736]
----------------------------

[0.83333333 1.03065599]
[0.37471264 1.14415083]
----------------------------

[1.         1.09121126]
[0.4045977  1.14040589]
----------------------------

[1.         1.44264147]
[0.42298851 0.9895104 ]
----------------------------

[1.         1.03243832]
[0.40689655 0.93208977]
----------------------------

[1.         0.93265324]
[0.43448276 0.90084747]
----------------------------

[1.         0.88659337]
[0.44827586 0.89930011]
----------------------------

[1.         1.04773156]
[0.45517241 0.76116124]
----------------------------

[1.         1.23380265]
[0.45517241 0.69502964]
----------------------------

[0.83333333 1.16696815]
[0.30229885 1.16976271]
----------------

[0.66666667 5.23774036]
[0.2816092 1.071607 ]
----------------------------

[0.83333333 3.32367088]
[0.32873563 1.08142037]
----------------------------

[0.83333333 4.87978608]
[0.37471264 1.09105938]
----------------------------

[0.83333333 4.93279936]
[0.40689655 1.08741846]
----------------------------

[0.66666667 3.3736237 ]
[0.42758621 0.96287616]
----------------------------

[1.         2.35383791]
[0.40689655 0.91386357]
----------------------------

[0.83333333 3.47794565]
[0.43678161 0.86573999]
----------------------------

[1.         3.82178977]
[0.44827586 0.85881465]
----------------------------

[1.         2.82541361]
[0.45517241 0.73664148]
----------------------------

[1.         1.78827097]
[0.45517241 0.6873818 ]
----------------------------

[0.66666667 7.02575366]
[0.3045977  1.08895188]
----------------------------

[0.83333333 7.3333334 ]
[0.33218391 1.00878105]
----------------------------

[0.5        3.24858281]
[0.36321839 1.01718983]
------------------